In [1]:
import os
import main

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/veronikagavrilova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
DATA_DIR = 'datasets/trec07p/data/'
LABELS_FILE = 'datasets/trec07p/full/index'
TRAINING_SET_RATIO = 0.5

In [3]:
labels = {}
# чтение меток
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()
        label, key = line.split()
        labels[key.split('/')[-1]] = 1 if label.lower() == 'ham' else 0

In [6]:
# Функция для чтения всех файлов почты в текстовом формате
def read_email_files():
    X = []
    y = []
    for i in range(len(labels)):
        filename = 'inmail.' + str(i+1)
        email_str = main.extract_email_text(
            os.path.join(DATA_DIR, filename))
        X.append(email_str)
        y.append(labels[filename])
    return X, y

In [7]:
X, y = read_email_files()

In [8]:
# Разделение набора данных на тренировочное и тестовое подмножество случайным образом
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, idx_train, idx_test = \
    train_test_split(X, y, range(len(y)),
    train_size=TRAINING_SET_RATIO, random_state=2)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Инициализация классификатора и предсказание меток
mnb = MultinomialNB()
mnb.fit(X_train_vector, y_train)
y_pred = mnb.predict(X_test_vector)

print(classification_report(y_test, y_pred, target_names=['Spam', 'Ham']))
print('Classification хaccuracy {:.1%}'.format(accuracy_score(y_test, y_pred)))

              precision    recall  f1-score   support

        Spam       0.99      0.94      0.96     25053
         Ham       0.89      0.98      0.93     12657

    accuracy                           0.95     37710
   macro avg       0.94      0.96      0.95     37710
weighted avg       0.96      0.95      0.95     37710

Classification accuracy 95.3%
